# ReferenceError: buildKey is not defined — 除錯筆記本

這份筆記本用來重現錯誤、定位作用域問題，並提供修正與測試策略。

## 1. 重現錯誤與最小案例 (loadCatalog + buildKey)

下面用最小範例模擬 `loadCatalog` 中呼叫 `buildKey` 時的錯誤堆疊。

In [ ]:
// 最小重現：buildKey 未定義
function loadCatalogMock(items) {
  items.forEach((item) => {
    // 這裡會拋出 ReferenceError
    const key = buildKey(item.item, item.cardNo);
    console.log('key:', key);
  });
}

try {
  loadCatalogMock([{ item: 'Topps Now', cardNo: '001' }]);
} catch (err) {
  console.error(err.name + ': ' + err.message);
  console.error(err.stack.split('\n').slice(0, 3).join('\n'));
}

## 2. 檢查作用域與載入順序 (函式提升/模組化)

`buildKey` 需要和 `loadCatalog` 同一個 `<script>` 作用域，或在模組中明確匯出與匯入。

In [ ]:
// ✅ 同一作用域：函式宣告在 loadCatalog 之前或之後都可以（函式提升）
function buildKey(item, cardNo) {
  return String(item || '').trim().toLowerCase() + '||' + String(cardNo || '').trim().toLowerCase();
}

function loadCatalogScoped(items) {
  return items.map((item) => buildKey(item.item, item.cardNo));
}

console.log(loadCatalogScoped([{ item: 'Topps Now', cardNo: '001' }]));

// ✅ 模組化：顯式匯出/匯入（示意）
// export function buildKey(...) { ... }
// import { buildKey } from './key-utils.js'

## 3. 新增防呆與降級策略 (安全預設 key)

在呼叫 `buildKey` 之前加上檢查，若缺失則降級為安全 key，避免整段流程中斷。

In [ ]:
function safeBuildKey(item, cardNo) {
  if (typeof buildKey === 'function') {
    return buildKey(item, cardNo);
  }
  // 降級策略：避免整體流程中斷
  return String(item || '').trim().toLowerCase() + '||' + String(cardNo || '').trim().toLowerCase();
}

function loadCatalogSafe(items) {
  return items.map((item) => safeBuildKey(item.item, item.cardNo));
}

console.log(loadCatalogSafe([{ item: 'Topps Now', cardNo: '001' }]));

## 4. 單元測試: buildKey 與 loadCatalog 關鍵流程

簡單測試案例，確認 `buildKey` 正常與缺失時不會中斷 `loadCatalog`。

In [ ]:
function assertEqual(actual, expected, label) {
  if (actual !== expected) {
    throw new Error(label + ': expected=' + expected + ', actual=' + actual);
  }
}

// 測試 buildKey 正常
function buildKeyTest(item, cardNo) {
  return String(item || '').trim().toLowerCase() + '||' + String(cardNo || '').trim().toLowerCase();
}
assertEqual(buildKeyTest('Topps', '001'), 'topps||001', 'buildKey basic');

// 測試 loadCatalog 不因缺失 buildKey 中斷
function safeBuildKeyTest(item, cardNo) {
  if (typeof buildKeyMissing === 'function') {
    return buildKeyMissing(item, cardNo);
  }
  return String(item || '').trim().toLowerCase() + '||' + String(cardNo || '').trim().toLowerCase();
}

function loadCatalogTest(items) {
  return items.map((item) => safeBuildKeyTest(item.item, item.cardNo));
}

const result = loadCatalogTest([{ item: 'Topps', cardNo: '001' }]);
assertEqual(result[0], 'topps||001', 'loadCatalog fallback');

console.log('✅ 測試通過');